In [1]:
using Revise, DynamicHMC, PosteriorDB, Random, StanLogDensityProblems, JSON, BridgeStan, DataFrames, LinearAlgebra, StatsBase, Distributions
using Plots, ColorSchemes, LaTeXStrings, Statistics, CSV, AdvancedHMC, DynamicObjects
Plots.theme(:default)
const BS = BridgeStan;

In [88]:
pdb = PosteriorDB.database()
model = PosteriorDB.model(pdb, "eight_schools_noncentered")
post = PosteriorDB.posterior(pdb, "eight_schools-eight_schools_noncentered")
impl = PosteriorDB.implementation(model, "stan")
path = PosteriorDB.path(impl)
data = PosteriorDB.dataset(post)
s = PosteriorDB.load(data, String);
bs_model = BS.StanModel(stan_file=path, data=s);
ref = PosteriorDB.reference_posterior(post)
df=DataFrame(PosteriorDB.load(ref));

In [178]:
lp = StanProblem(bs_model);
D = 10;
initial_θ = rand(D);
n_samples, n_adapts = 8000, 2000;
metric = DiagEuclideanMetric(D);
hamiltonian = Hamiltonian(metric, lp);
initial_ϵ = find_good_stepsize(hamiltonian, initial_θ);
integrator = Leapfrog(initial_ϵ);
proposal = NUTS{MultinomialTS, GeneralisedNoUTurn}(integrator);
adaptor = StanHMCAdaptor(MassMatrixAdaptor(metric), StepSizeAdaptor(0.8, integrator));
samples, stats = sample(hamiltonian, proposal, initial_θ, n_samples, adaptor, n_adapts; progress=true);

Sampling   0%|                               |  ETA: 0:12:02
  iterations:                                   2
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  0.5
  n_steps:                                      1
  is_accept:                                    true
  acceptance_rate:                              0.0
  log_density:                                  -9.008526251177598
  hamiltonian_energy:                           12.708457112487306
  hamiltonian_energy_error:                     0.0
  max_hamiltonian_energy_error:                 4.275140685799279e14
  tree_depth:                                   0
  numerical_error:                              true
  step_size:                                    4.6305929745559276
  nom_step_size:                                4.6305929745559276
  is_adapt:                                     true



















Sampling 100%|███████████████████████████████| Time: 0:00:00
  iterations:                                   8000
  ratio_divergent_transitions:                  0.0
  ratio_divergent_transitions_during_adaption:  0.0
  n_steps:                                      7
  is_accept:                                    true
  acceptance_rate:                              0.9996631866288145
  log_density:                                  -7.883221140637642
  hamiltonian_energy:                           12.261434336830948
  hamiltonian_energy_error:                     -0.10168988877392415
  max_hamiltonian_energy_error:                 -0.36220975447912984
  tree_depth:                                   3
  numerical_error:                              false
  step_size:                                    0.5139431750108344
  nom_step_size:                                0.5139431750108344
  is_adapt:                                     false
  mass_matrix:                 

┌ Info: Finished 8000 sampling steps for 1 chains in 0.62266691 (s)
│   h = Hamiltonian(metric=DiagEuclideanMetric([0.9342838452400835, 0.8845 ...]), kinetic=GaussianKinetic())
│   κ = HMCKernel{AdvancedHMC.FullMomentumRefreshment, Trajectory{MultinomialTS, Leapfrog{Float64}, GeneralisedNoUTurn{Float64}}}(AdvancedHMC.FullMomentumRefreshment(), Trajectory{MultinomialTS}(integrator=Leapfrog(ϵ=0.514), tc=GeneralisedNoUTurn{Float64}(10, 1000.0)))
│   EBFMI_est = 0.9935941009273499
│   average_acceptance_rate = 0.807194288588557
└ @ AdvancedHMC /home/meenaljhajharia/.julia/packages/AdvancedHMC/P0nla/src/sampler.jl:246


In [152]:
    constrained_draws = hcat([vcat(col...) for col in eachcol(df)]...)
    unc_sample_array = vcat([
            param_unconstrain(bs_model, collect(row))' for row in eachrow(constrained_draws)
        ]...);

In [170]:
using TransformVariables
t = as((μ = asℝ, σ = asℝ₊, τ = asℝ₊, θs = as(Array, 8)))
dimension(t)

11

In [176]:
P = TransformedLogDensity(t, lp)
∇P = ADgradient(:ForwardDiff, lp);

In [175]:
using TransformedLogDensities, LogDensityProblemsAD

In [180]:
samples

8000-element Vector{Vector{Float64}}:
 [-1.7003134669502395, -0.90671438636748, 0.7347831077096086, -1.1805525907692078, 0.25723581572030385, 0.0881677704977733, -1.6501416091039252, -1.0503802488939349, 1.5333279392074668, 0.29740362668411824]
 [-1.7003134669502395, -0.90671438636748, 0.7347831077096086, -1.1805525907692078, 0.25723581572030385, 0.0881677704977733, -1.6501416091039252, -1.0503802488939349, 1.5333279392074668, 0.29740362668411824]
 [-0.46051656023472665, 0.7556628794865422, 0.7109147446157704, 0.1139996564072373, 0.977664923749782, -0.5388752981146849, 0.33521653740148727, 0.3117257297494793, 1.7715940839440474, 1.4333327163299951]
 [0.7038792498446453, 0.09773494337493305, -0.23845752471278175, 0.46302577606516326, -1.9452182437208376, 0.5902361322409491, -0.7019118979872371, -0.5872691456146444, 2.4155624555359383, 0.15154362836173693]
 [-0.5335627597788599, 0.2067729362353954, 0.21328317341810832, -0.4844052125266387, 1.8277402548821382, -0.686849763290789, 1.164144